## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-17-15-49-00 +0000,wsj,Jefferies Shares Pare Losses After CEO Says Fi...,https://www.wsj.com/opinion/jefferies-shares-p...
1,2025-10-17-15-48-11 +0000,nyt,Live Updates: Zelensky Will Meet Trump as He P...,https://www.nytimes.com/live/2025/10/17/us/tru...
2,2025-10-17-15-47-49 +0000,nyt,Trump Pushes Indiana Lawmakers to Redraw State...,https://www.nytimes.com/2025/10/17/us/politics...
3,2025-10-17-15-47-36 +0000,bbc,Shares recover after US banks cause a scare,https://www.bbc.com/news/articles/c4gjz82wx4wo...
4,2025-10-17-15-46-25 +0000,nyt,"How Would Putin, Who Faces an International Ar...",https://www.nytimes.com/2025/10/17/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,64
17,new,19
38,china,16
127,against,13
375,military,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
288,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,142
1,2025-10-17-15-48-11 +0000,nyt,Live Updates: Zelensky Will Meet Trump as He P...,https://www.nytimes.com/live/2025/10/17/us/tru...,138
344,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...,128
247,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,125
293,2025-10-16-20-34-59 +0000,nypost,Ex-Trump national security adviser John Bolton...,https://nypost.com/2025/10/16/us-news/ex-trump...,122


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
288,142,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
191,73,2025-10-17-03-46-08 +0000,nypost,"Zohran Mamdani dodges attacks, straight answer...",https://nypost.com/2025/10/16/us-news/mamdani-...
293,58,2025-10-16-20-34-59 +0000,nypost,Ex-Trump national security adviser John Bolton...,https://nypost.com/2025/10/16/us-news/ex-trump...
133,55,2025-10-17-10-00-00 +0000,wapo,"The average new-car price hit a record $50,000...",https://www.washingtonpost.com/business/2025/1...
369,53,2025-10-16-16-00-00 +0000,wsj,An extended government shutdown raises the pro...,https://www.wsj.com/economy/central-banking/fe...
299,48,2025-10-16-20-21-13 +0000,nypost,Thune says ‘time’s right’ for Russia sanctions...,https://nypost.com/2025/10/16/us-news/thune-sa...
90,47,2025-10-17-11-39-43 +0000,bbc,Zelensky to make case for missiles at White Ho...,https://www.bbc.com/news/articles/crmxz37nv3zo...
144,41,2025-10-17-09-23-40 +0000,nypost,Creepy footage shows suspect charged in beauty...,https://nypost.com/2025/10/17/us-news/philadel...
44,40,2025-10-17-14-23-12 +0000,nypost,Tattooed salon owner allegedly threatened to k...,https://nypost.com/2025/10/17/us-news/bat-cave...
371,36,2025-10-16-15-57-08 +0000,nyt,Israel Marks Two Years Since Oct. 7 as a Fragi...,https://www.nytimes.com/2025/10/16/world/middl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
